In [1]:
# FIXME: remove this cd
%cd ..

/Users/gexahedron/devel/domblar


In [2]:
synth_count = 4

from domblar.domblar import Domblar

d = Domblar(
    synth_count=synth_count,
    context='dexed',
    mode='tracker',
)

In [6]:
from domblar.chord_theory import Chords
from domblar.tonal_theory import build_scale_from_steps

# TODO:
# ctx = Context(edo=22, scale='4333324')
# or maybe TODO: create global variable for context

def set_synths():
    d.set_synth(0, 'marimba20')
    d.set_synth(1, 'tub_bells')
    # d.set_synth(1, 'celeste')
    d.set_synth(2, 'celeste')
    d.set_synth(3, 'piano3')
set_synths()

dur = 0.15/1.2

edo = 22
scale = build_scale_from_steps('4333324', edo)

d <[
    Chords()  # TODO: Chords(ctx)
    .synths([0, 1, 2, 3])
    .seq(4, [-3, 2],
        #  TODO:
        #  mods='r1:4/4 f6:5/6',  # r=rewrite, f=modulate_from
         length=5,
         edo=edo,
         scale=scale
         )
    .voicing('0_ 1 2_')
    .q('quartal')
    .g([
        '0:4*2',
        '3__:2 .:4 2__:2',
        '1 . 1 . 1 . 1 .',  # TODO: '{1 .}*4',
        '. 2 . 3 . 2 . 3',  # TODO: '{. 2 . 3}*2',
        ])
    .dur(dur)  # FIXME: better to replace with beat-dur
    .sus(dur * 0.95)
    .once()
]


voices.events [[(-12, 4), (-12, 4), (-12, 4), (-12, 4), (-9, 4), (-9, 4), (-4, 4), (-4, 4), (0, 4), (0, 4)], [(-6, 2), ('.', 4), (-37, 2), (-4, 2), ('.', 4), (-37, 2), (-6, 2), ('.', 4), (-37, 2), (-6, 2), ('.', 4), (-37, 2), (-9, 2), ('.', 4), (-34, 2)], [(18, 1), ('.', 1), (18, 1), ('.', 1), (18, 1), ('.', 1), (18, 1), ('.', 1), (22, 1), ('.', 1), (22, 1), ('.', 1), (22, 1), ('.', 1), (22, 1), ('.', 1), (26, 1), ('.', 1), (26, 1), ('.', 1), (26, 1), ('.', 1), (26, 1), ('.', 1), (26, 1), ('.', 1), (26, 1), ('.', 1), (26, 1), ('.', 1), (26, 1), ('.', 1), (26, 1), ('.', 1), (26, 1), ('.', 1), (26, 1), ('.', 1), (26, 1), ('.', 1)], [('.', 1), (7, 1), ('.', 1), (38, 1), ('.', 1), (7, 1), ('.', 1), (38, 1), ('.', 1), (7, 1), ('.', 1), (40, 1), ('.', 1), (7, 1), ('.', 1), (40, 1), ('.', 1), (7, 1), ('.', 1), (38, 1), ('.', 1), (7, 1), ('.', 1), (38, 1), ('.', 1), (7, 1), ('.', 1), (38, 1), ('.', 1), (7, 1), ('.', 1), (38, 1), ('.', 1), (10, 1), ('.', 1), (35, 1), ('.', 1), (10, 1), ('.', 1)

In [5]:
d.tracker.stop()

In [5]:
import random
from collections import defaultdict

from domblar.music_math import ratio_to_floats, C4_FREQ
from domblar.transformations import transpose


# FIXME: unused, but helpful
def l_system_stats(s):
    counts = defaultdict(int)
    max_runs = defaultdict(int)
    cur_run = 0
    prev_char = ''
    for c in s:
        counts[c] += 1
        if c == prev_char:
            cur_run += 1
        else:
            if prev_char:
                max_runs[prev_char] = max(max_runs[prev_char], cur_run)
            cur_run = 1
        prev_char = c
    max_runs[prev_char] = max(max_runs[prev_char], cur_run)
    return {'counts': counts.items(), 'max_runs': max_runs.items()}


# FIXME: rename
def run_l_system(rules, start, iterations):
    cur = start
    for c in list(rules.keys()):
        if type(rules[c]) is not list:
            rules[c] = [rules[c]]
    for _ in range(iterations):
        c_counter = defaultdict(int)
        rewritten = ''
        for c in cur:
            rewritten += rules[c][c_counter[c] % len(rules[c])]
            c_counter[c] += 1
        # FIXME
        # print(f'{i + 1}: ', rewritten.startswith(cur), l_system_stats(rewritten))
        cur = rewritten
    return cur


# FIXME: hide defaults for voices and quality
# FIXME: usually default for voices is 3
def tonnetz(scale, edo, degree=1, mj=0, mn=0, voices=4, quality='tertian', voicing=None):
    # min3 = ratio_to_floats(6, 5)
    # maj3 = ratio_to_floats(5, 4)

    # FIXME: this is for 11edo
    min3 = ratio_to_floats(11, 9)
    maj3 = ratio_to_floats(9, 7)

    shift = round(edo * maj3) * mj + round(edo * min3) * mn
    return chord_in_edo(transpose(scale, shift), edo, degree,
                        voices=voices, voicing=voicing, quality=quality)


# FIXME: write out assumptions about scales somewhere
# that 0 corresponds to C4
# and make it configurable
def freq_to_edo(freq, edo):
    return round(edo * ratio_to_floats(freq, C4_FREQ))


def recalc_chord(chord, prev_edo, new_edo):
    new_chord = []
    for n in chord:
        freq = get_freq(n, list(range(prev_edo)), prev_edo)
        new_chord.append(freq_to_edo(freq, new_edo))
    return new_chord

def add_scale_and_edo(progression, scale, edo):
    result = []
    for degree in progression:
        result.append((degree, scale, edo))
    return result


In [6]:
last_chord = None
last_edo = None
# TODO: last scale

def set_synths():
    d.set_synth(0, 'marimba20')
    d.set_synth(1, 'tub_bells')
    # d.set_synth(1, 'celeste')
    d.set_synth(2, 'celeste')
    d.set_synth(3, 'piano3')
set_synths()

In [14]:
# t.modulate_synth(60, 10, 1,
#                  'tub_bells', 'harpsich1')

In [90]:
last_chord = None

def set_synths():
    for i in range(4):
        if i == 0:
            t.d.set_synth(i, 'big_grand')
        else:
            t.d.set_synth(i, 'epiano1')
set_synths()

In [95]:
# part B
# NRT Tonnetz progressions
# PLR cycle
# L-systems for rhythm
# also in the spirit of Sergei Zagny - Sonata

rules = {
    '1': ['12', '112'],
    '2': ['221', '21']
}
l_rhythm = run_l_system(rules, '2', 9)

# Porcupine bright harmonic 11th mode - Porcupine[7] 6|0 b7
edo, steps = 15, '3222213'

steps = [int(x) for x in steps]
scale = build_scale_from_steps(steps, edo)

once = False
use_gestures = False
# use_gestures = True
# final_round = False
final_round = True

gestures = None
if use_gestures:
    gestures = [
        '0 . 3 .',
        '. 1 . 0',
        '1 . 2 .',
        '. 2 . 3'
    ]


def build_chord_loop(scale, edo, degree=1):
    voicing = '0_ 2_'
    chord_loop = []
    chord_loop.append(tonnetz(scale, edo, mn=-1, mj=0, degree=degree, voicing=voicing,
                              quality='add6'))
    chord_loop.append(tonnetz(scale, edo, degree=degree, voicing=voicing,
                              quality='add6'))
    chord_loop.append(tonnetz(scale, edo, mn=0, mj=1, degree=degree, voicing=voicing,
                              quality='add2'))
    # chord_loop.append(tonnetz(scale, edo, mn=0, mj=2, degree=degree, voicing=voicing,
    #                           quality='add2'))
    # chord_loop.append(tonnetz(scale, edo, mn=-1, mj=2, degree=degree, voicing=voicing,))
    chord_loop = cycle(chord_loop)
    return chord_loop

chord_loop = build_chord_loop(scale, edo)

from domblar.transformations import shift

# build chords
chords = [chord_loop[-1]]
if last_chord:
    chords = [last_chord]

# loop_count = 1
loop_count = 4
inblock_reps = 1
block_reps = 1

# loop_count = 3
# inblock_reps = 3
# block_reps = 3

for _ in range(loop_count):
    for _ in range(block_reps):
        for rep in range(inblock_reps):
            for chord in chord_loop:
                chords.append(cadence_in_edo(chords[-1], chord, edo))
        steps = shift(steps, 1)
        steps = [int(x) for x in steps]
        scale = build_scale_from_steps(steps, edo)
        chord_loop = build_chord_loop(scale, edo)

muls = [1]
if last_chord is not None:
    muls = []
r_idx = 0

muls_version = 2  # using l-rhythms
# muls_version = 1

if muls_version == 1:
    for _ in range(loop_count):
        for rep in range(inblock_reps):
            for chord in chord_loop:
                if (r_idx % (len(chord_loop) - 1) != 0):
                    muls.append(1)
                else:
                    muls.append(2)
                r_idx += 1
else:
    for _ in range(loop_count):
        sub_muls = []
        for rep in range(inblock_reps):
            for chord in chord_loop:
                sub_muls.append(int(l_rhythm[r_idx]))
                r_idx += 1
        muls.extend(sub_muls * block_reps)

if final_round:
    chords = chords[:1]
    for chord in chord_loop[:2]:
        chords.append(cadence_in_edo(chords[-1], chord, edo))
    muls = [1, 15, 16]
    once = True
    use_gesture = False


# some post-processing
chords = chords[:len(muls)]
loop_from = 1
if last_chord is not None:
    loop_from = 0
    chords = chords[1:]
    muls = muls[1:]


voices_with_dur = chords_to_voices(chords, durs=muls,
                                   gestures=gestures, edo=edo
)

dur = 0.25
if use_gestures:
    dur = 0.25/3

if t.queue(
    voices=voices_with_dur,
    edo=edo,
    dur=dur,
    sus=dur * 0.7,
    loop_from=loop_from,
    once=once,
):
    last_chord = chords[-1]


Queued!


In [17]:
t.stop()

In [96]:
last_chord = None
last_edo = None

def set_synths():
    t.d.set_synth(0, 'celeste')
    t.d.set_synth(1, 'epiano1')
    t.d.set_synth(2, 'celeste')
    t.d.set_synth(3, '*vibration')
set_synths()


In [102]:
# part C==A'
# decatonic scales (tertian) harmony
# 12edo, 15edo, 17edo, 22edo

# once = False
once = True
use_gestures = False
# use_gestures = True

# edo, steps = 12, '1121111211'  # Four-semitone tritone scale/Messiaen's 7th "mode"
# edo, steps = 15, '2121212112'  # Blackwood decatonic
# edo, steps = 17, '2122122212'  # neutrominant
edo, steps = 22, '3133131331'  # astrology[10] mode
# edo, steps = 22, '2232222322'  # pajara[10] symmetric decatonic

# choose random mode
steps = [int(x) for x in steps]
steps = shift(steps, random.randint(0, 9))
scale = build_scale_from_steps(steps, edo=edo, start=-edo)

# generate random sequence
x = random.randint(1, 10)
y = -x + random.randint(1, 9)
z = random.randint(2, 10)
seq = [x, y]
notes = 12
dup_start = False
z = 5; seq = [1, -2, -1, 1]; notes = 31; dup_start = True

progression = cycle(sequence(z, seq,
                              scale=scale, edo=edo,
                              notes=notes,
                              rewrite={1: (5, 6)},
), dup_start=dup_start) + [('1j', scale, edo)]


muls_for_3rd_part = []
stop_dur = 6
last_dur = 1
for idx, deg in enumerate(progression):
    if deg in [1, '1j']:
        muls_for_3rd_part.append(stop_dur)
        last_dur = stop_dur
        # stop_dur += 2
    else:
        muls_for_3rd_part.append(last_dur)
        last_dur = max(last_dur // 2, 1)

muls_for_3rd_part[-1] = stop_dur * 2

if last_chord is not None and not (-edo * 3 < min(last_chord) < 0):
    last_chord = None
if last_chord is not None:
    last_chord = recalc_chord(last_chord, last_edo, edo)

chords = progress(
    progression,
    voicing='1^',
    start_from=last_chord,
    voices_default=4,
    always_jump_on=[1],
)
print(progression)

gestures = None
if use_gestures:
    gestures = [
        '0:2',
        '1_ 2',
        '2_ 1',
        '3:2',
    ]

dur = 0.15
dur = 0.15/2
# dur = 0.15/3
if use_gestures:
    dur = dur/2

voices_with_dur = chords_to_voices(chords,
                                   gestures=gestures, edo=edo,
                                   durs=muls_for_3rd_part)

if t.queue(
    voices=voices_with_dur,
    edo=edo,
    dur=dur,
    sus=dur * 0.95,
    loop_from=1,
    once=once,
):
    last_chord = chords[-1]
    last_edo = edo

[(5, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (6, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (4, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (3, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (4, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (5, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (3, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (2, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (3, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (4, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (2, [-22, -19, -16, -15, -12, -11, -8, -5, -4, -1], 22), (5, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (6, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (7, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (5, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (4, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (5, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (6, [11, 14, 17, 18, 21, 22, 25, 28, 29, 32], 22), (4, [11, 14, 17

In [167]:
t.stop()

In [45]:
finetuning = 0
synth_idx = 1

try:
    finetuning
except Exception as e:
    t.d.open_editor(synth_idx)
    finetuning = 1
else:
    if finetuning == 0:
        t.d.open_editor(synth_idx)
    elif finetuning == 1:
        # transfer instrument to all synths
        t.d.save_preset(synth_idx)
        import time
        time.sleep(0.5)
        for i in range(synth_count):
            t.d.load_preset(i)
    else:
        t.d.print_params(synth_idx)

In [12]:
# cleanup
t.d.stop_server()

In [18]:
last_chord = None
last_edo = None
# TODO: last scale

def set_synths():
    for i in range(4):
        t.d.set_synth(i, 'epiano1')
    # t.d.set_synth(0, 'marimba20')
set_synths()

In [82]:
# edo, steps = 12, '2212221'
edo, steps = 22, '4333324'  # Porcupine bright harmonic 11th mode - Porcupine[7] 6|0 b7
scale = build_scale_from_steps(steps, edo=edo, start=0)

last_chord = None

progression = sequence(1, [1],
    scale=scale, edo=edo,
    modulate_from={
        2: (5, 2),
        3: (5, 3),
        4: (5, 4),
        5: (5, 5),
        6: (5, 6),
    },
    notes=13)

for i in range(len(progression)):
    if progression[i][1] == scale:
        progression[i] = (f'{progression[i][0]}j', progression[i][1], progression[i][2])
    else:
        progression[i] = (f'{progression[i][0]}add2', progression[i][1], progression[i][2])

print(progression)

chords = progress(progression,
    # voicing='0_ 2_',
    voicing='1^',
    voices_default=4,
    start_from=last_chord,
    # quality='quartal',
    # quality='tertian',
    # quality='add2',
    # quality='add6',
)

for chord in chords:
    print([chord[0], chord[2], chord[3], chord[1]])

from domblar.transformations import split

gestures = [
    # '0_:4x2',
    # '.:2 0:2 .:2 1:2',
    # '0_:2 .:1 0:2 .:1 0_:2',
    '0_:2 2:1 0:2 1:1 0_:2',

    # '3_:2 .:4 2_:2',
    '3__:2 .:2 2__:2 .:2',
    # '3__:2 .:2 2_:1 3__:3',

    *split(['1',2,'1','3'] * 2),
]

voices_with_dur = chords_to_voices(chords,
                                #    gestures=gestures, edo=edo
)

dur = 0.5/4
dur = 0.5
if t.queue(
    voices=voices_with_dur,
    edo=edo,
    dur=dur,
    sus=dur * 0.95,
    once=True,
):
    last_chord = chords[-1]
    last_edo = edo

[('1j', [0, 4, 7, 10, 13, 16, 18], 22), ('5add6', [4, 8, 11, 14, 17, 20, 22], 22), ('2j', [0, 4, 7, 10, 13, 16, 18], 22), ('5add6', [7, 11, 14, 17, 20, 23, 25], 22), ('3j', [0, 4, 7, 10, 13, 16, 18], 22), ('5add6', [10, 14, 17, 20, 23, 26, 28], 22), ('4j', [0, 4, 7, 10, 13, 16, 18], 22), ('5add6', [13, 17, 20, 23, 26, 29, 31], 22), ('5j', [0, 4, 7, 10, 13, 16, 18], 22), ('5add6', [16, 20, 23, 26, 29, 32, 34], 22), ('6j', [0, 4, 7, 10, 13, 16, 18], 22), ('7j', [0, 4, 7, 10, 13, 16, 18], 22), ('1j', [0, 4, 7, 10, 13, 16, 18], 22)]
[0, 13, 18, 29]
[0, 11, 17, 30]
[4, 10, 16, 22]
[3, 11, 14, 20]
[7, 4, 13, 18]
[6, 1, 14, 17]
[10, 0, 7, 16]
[9, -2, 4, 17]
[13, -4, 4, 10]
[12, -2, 1, 7]
[16, -9, 0, 7]
[18, -6, 4, 10]
[22, -4, 7, 13]
Queued!


In [101]:
t.stop()